# Задание 1. готово

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:
KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [1]:
KEYWORDS = ['python', 'парсинг']

In [3]:
from bs4 import BeautifulSoup
import time
import requests

In [4]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')

In [107]:
posts = soup.find_all('article', class_='post')

In [108]:
for post in posts:
    hubs = post.find_all('a', class_='hub-link')
    for hub in hubs:
            hub_lower = hub.text.lower()
            for key in KEYWORDS:
                if hub_lower == key:
                    title_element = post.find('a', class_='post__title_link')
                    data_element = post.find('span', class_='post__time')
                    print(f"<{data_element.text}> - <{title_element.text}> - <{title_element.attrs.get('href')}>")

<сегодня в 10:09> - <Telegram-бот на Python для создания задач в MS Outlook и заметок в Evernote> - <https://habr.com/ru/post/549408/>


# Задание 2.

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [103]:
import json

In [104]:
import pandas as pd

In [113]:
# Получаю данные (ему вечно не хватало headers, так что я просто добавил все):

URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
data = {
    "emailAddresses":["007.777_95@mail.ru"]
}
headers = {
    'Vaar-Version': '0',
    'Content-Type': 'application/json;charset=utf-8',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'ru',
    'Accept-Encoding': 'gzip, deflate, br',
    'Host': 'identityprotection.avast.com',
    'Origin': 'https://www.avast.com',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15',
    'Referer': 'https://www.avast.com/hackcheck/',
    'Content-Length': '34',
    'Connection': 'keep-alive',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
    'Vaar-Version': '0',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Vaar-Header-App-Build-Version': '1.0.0'
}
    
time.sleep(0.3)
req = requests.post(URL, json = data , headers = headers)

In [114]:
# Перевел все в нормальный вид:
time.sleep(0.3)
data = json.loads(req.text)

In [115]:
# Пробую добавить это все в датафрейм:

keys = data['breaches'].keys()
site = []
date = []
descr = []
mail = []
for i in keys:
    site.append(data['breaches'][i]['site'])
    date.append(data['breaches'][i]['publishDate'])
    descr.append(data['breaches'][i]['description'])
    mail.append('temishe@gmail.com')

df = pd.DataFrame({'почта': mail, 'дата утечки': date, 'источник утечки': site, 'описание утечки': descr})

TypeError: 'NoneType' object is not subscriptable

In [112]:
# Собираю это всё в функцию:

def check_email():
    EMAIL = ['temishe77@gmail.com', '007.777_95@mail.ru']
    site = []
    date = []
    descr = []
    mail = []
    for email in EMAIL:
        time.sleep(0.3)
        URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
        data = {
            "emailAddresses":[email]
        }
        headers = {
        'Vaar-Version': '0',
        'Content-Type': 'application/json;charset=utf-8',
        'Accept': 'application/json, text/plain, */*',
        'Accept-Language': 'ru',
        'Accept-Encoding': 'gzip, deflate, br',
        'Host': 'identityprotection.avast.com',
        'Origin': 'https://www.avast.com',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15',
        'Referer': 'https://www.avast.com/hackcheck/',
        'Content-Length': '34',
        'Connection': 'keep-alive',
        'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
        'Vaar-Version': '0',
        'Vaar-Header-App-Product': 'hackcheck-web-avast',
        'Vaar-Header-App-Build-Version': '1.0.0'
        }

        req = requests.post(URL, json = data , headers = headers)
        
        data = json.loads(req.text)
        
        keys = data['breaches'].keys()
        
        for i in keys:
            site.append(data['breaches'][i]['site'])
            date.append(data['breaches'][i]['publishDate'])
            descr.append(data['breaches'][i]['description'])
            mail.append(email)

    df = pd.DataFrame({'почта': mail, 'дата утечки': date, 'источник утечки': site, 'описание утечки': descr})
    return df

check_email()

,почта,дата утечки,источник утечки,описание утечки
0,temishe77@gmail.com,2020-07-23T00:00:00Z,wattpad.com,"In June 2020, the online writing community Wat..."
1,temishe77@gmail.com,2019-06-13T00:00:00Z,canva.com,"In May 2019, graphic-design site Canva's datab..."
2,temishe77@gmail.com,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
3,temishe77@gmail.com,2021-02-04T00:00:00Z,rzd-bonus.ru,"In June 2020, the Russian Railways Bonus Progr..."
4,007.777_95@mail.ru,2016-10-21T00:00:00Z,myspace.com,"Shortly before the 2016 Memorial Day weekend, ..."
5,007.777_95@mail.ru,2017-11-04T00:00:00Z,myheritage.com,"In October 2017, a customer database belonging..."
6,007.777_95@mail.ru,2017-02-14T00:00:00Z,parapa.mail.ru,"In July and August 2016, two criminals execute..."
7,007.777_95@mail.ru,2017-06-14T00:00:00Z,nnm-club.me,"In September 2013, Russian torrent tracker NNM..."
8,007.777_95@mail.ru,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
